# Finding interval Estimators

In [27]:
import  numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

# 1. Carregando data.csv

data = pd.read_csv('Data_CSV/data_Breast_Cancer_Wisconsin.csv')
# Breast_cancer_data_Suwal_2018.csv
# Data_Alzheimer_Diagnosis.csv
# data_Breast_Cancer_Wisconsin.csv
# data_Mammography.csv

from sklearn.impute import SimpleImputer

# Initialize the imputer
imputer = SimpleImputer(strategy='mean')

# Fit and transform the data
data_filled = imputer.fit_transform(data)
data_filled = pd.DataFrame(data_filled, columns=data.columns)

data = data_filled

# Supondo que a última coluna seja a classe
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values


print(X.shape)
print(y[len(y)==-1].shape)

# Converta para DataFrame para facilitar a seleção de características
# df = pd.DataFrame(X)
# corr_matrix = df.corr().abs()

# Encontre características altamente correlacionadas
# upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
# to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]  # Limite

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Remova as mesmas características de ambos os conjuntos (treinamento e teste)
# X_train_reduced = X_train.drop(to_drop, axis=1)
# X_test_reduced = X_test.drop(to_drop, axis=1)

# print(f"Shape of X_train_reduced: {X_train_reduced.shape}")
# print(f"Shape of X_test_reduced: {X_test_reduced.shape}")
# print(f"Shape of y_train: {y_train.shape}")
# print(f"Shape of y_test: {y_test.shape}")
print(f"Total positive cases in the dataset: {sum(y == 1)}")


(699, 9)
(0, 699)
Total positive cases in the dataset: 241


In [28]:
from sklearn.preprocessing import StandardScaler  # Adicione essa linha para importar StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA


# # Normalizar os dados
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# #PCA analysis
# pca = PCA(n_components=0.95)  # Preserve 95% of the variance
# X_train_scaled = pca.fit_transform(X_train_scaled)
# X_test_scaled = pca.transform(X_test_scaled)
print(f"Positive cases in the training set: {sum(y_train == 1)}")
print(f"Positive cases in the test set: {sum(y_test == 1)}")
# Inicializar o Bayesiano
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)
nb_predictions = nb_classifier.predict(X_test)

# Inicializar o LDA com shrinkage para reduzir a colinearidade
lda_classifier = LinearDiscriminantAnalysis(solver='eigen', shrinkage='auto')
lda_classifier.fit(X_train, y_train)
lda_predictions = lda_classifier.predict(X_test)

# Quadrado
qda_classifier = QuadraticDiscriminantAnalysis(reg_param=0.1)
qda_classifier.fit(X_train, y_train)
qda_predictions = qda_classifier.predict(X_test)

Positive cases in the training set: 217
Positive cases in the test set: 24


In [15]:
def evaluate_with_confusion_matrix(name, y_true, y_pred):
        cm = confusion_matrix(y_true, y_pred)
        print(f'\n{name} Classifier - Confusion Matrix:')
        print(cm)

    # Acurácia por classe
        class_accuracies = cm.diagonal() / cm.sum(axis = 1)
        print(f'Acurácia por classe:{class_accuracies}')

    # Acurácia geral
        overall_accuracy = cm.diagonal().sum() / cm.sum()
        print(f'Acurácia geral: {overall_accuracy:.4f}')
        print(classification_report(y_true, y_pred))

evaluate_with_confusion_matrix('Naive Bayes',y_test, nb_predictions)
evaluate_with_confusion_matrix('Linear Discriminant Analysis', y_test, lda_predictions)
evaluate_with_confusion_matrix('Quadratic Discriminant Analysis', y_test, qda_predictions)


Naive Bayes Classifier - Confusion Matrix:
[[1048   47]
 [   5   19]]
Acurácia por classe:[0.95707763 0.79166667]
Acurácia geral: 0.9535
              precision    recall  f1-score   support

          -1       1.00      0.96      0.98      1095
           1       0.29      0.79      0.42        24

    accuracy                           0.95      1119
   macro avg       0.64      0.87      0.70      1119
weighted avg       0.98      0.95      0.96      1119


Linear Discriminant Analysis Classifier - Confusion Matrix:
[[1077   18]
 [  11   13]]
Acurácia por classe:[0.98356164 0.54166667]
Acurácia geral: 0.9741
              precision    recall  f1-score   support

          -1       0.99      0.98      0.99      1095
           1       0.42      0.54      0.47        24

    accuracy                           0.97      1119
   macro avg       0.70      0.76      0.73      1119
weighted avg       0.98      0.97      0.98      1119


Quadratic Discriminant Analysis Classifier - Confusi

In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Carregar os dados do arquivo CSV
df = pd.read_csv('Data_CSV/data_Mammography.csv')

# Supondo que a última coluna seja a variável alvo (classe)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Converter rótulos para valores numéricos, se necessário
if y.dtype == 'object':
    y = LabelEncoder().fit_transform(y)

# Criar o modelo Discriminante Linear
model = LinearDiscriminantAnalysis()
print('\n\n\n\n Linear')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 10  # Número de iterações
test_size = 0.1  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())

model = LinearDiscriminantAnalysis()
print('\n\n\n\n Quadrático')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 10  # Número de iterações
test_size = 0.1  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())



model = GaussianNB()
print('\n\n\n\n Naive Bayes')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 10  # Número de iterações
test_size = 0.1  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())





 Linear
Acurácia por repetição: [0.97587131 0.97944593 0.97944593 0.97676497 0.974084   0.98033959
 0.97229669 0.98391421 0.96782842 0.97587131]
Média da acurácia: 0.9765862377122432
Desvio padrão da acurácia: 0.00431923001737072




 Quadrático
Acurácia por repetição: [0.97676497 0.97319035 0.98302055 0.98123324 0.97229669 0.98302055
 0.97765862 0.97855228 0.97944593 0.97497766]
Média da acurácia: 0.9780160857908846
Desvio padrão da acurácia: 0.003601330058911509




 Naive Bayes
Acurácia por repetição: [0.95352994 0.96336014 0.95084897 0.9562109  0.95889187 0.94906166
 0.95799821 0.95263628 0.96336014 0.95889187]
Média da acurácia: 0.9564789991063452
Desvio padrão da acurácia: 0.00467869172929703


In [34]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Carregar os dados do arquivo CSV
data = pd.read_csv('Data_CSV/Breast_cancer_data_Suwal_2018.csv')

# Supondo que a última coluna seja a variável alvo (classe)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Converter rótulos para valores numéricos, se necessário
if y.dtype == 'object':
    y = LabelEncoder().fit_transform(y)

# Criar o modelo Discriminante Linear
model = LinearDiscriminantAnalysis()
print('\n\n\n\n Linear')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 10  # Número de iterações
test_size = 0.1  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())

model = LinearDiscriminantAnalysis()
print('\n\n\n\n Quadrático')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 10  # Número de iterações
test_size = 0.1  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())



model = GaussianNB()
print('\n\n\n\n Naive Bayes')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 10  # Número de iterações
test_size = 0.1  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())







 Linear
Acurácia por repetição: [0.94285714 0.92857143 0.92857143 0.91428571 0.9        0.98571429
 0.97142857 0.98571429 0.95714286 0.98571429]
Média da acurácia: 0.95
Desvio padrão da acurácia: 0.030135747299612853




 Quadrático
Acurácia por repetição: [0.97142857 0.98571429 0.98571429 0.94285714 0.95714286 0.94285714
 0.91428571 0.97142857 0.94285714 0.98571429]
Média da acurácia: 0.96
Desvio padrão da acurácia: 0.02285714285714288




 Naive Bayes
Acurácia por repetição: [0.94285714 0.91428571 0.97142857 0.92857143 0.95714286 0.97142857
 0.98571429 0.98571429 0.95714286 0.98571429]
Média da acurácia: 0.96
Desvio padrão da acurácia: 0.0237332110369088


In [33]:


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Carregar os dados do arquivo CSV
data = pd.read_csv('Data_CSV/Data_Alzheimer_Diagnosis.csv')


# Supondo que a última coluna seja a variável alvo (classe)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Converter rótulos para valores numéricos, se necessário
if y.dtype == 'object':
    y = LabelEncoder().fit_transform(y)

# Criar o modelo Discriminante Linear
model = LinearDiscriminantAnalysis()
print('\n\n\n\n Linear')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 10  # Número de iterações
test_size = 0.1  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())

model = LinearDiscriminantAnalysis()
print('\n\n\n\n Quadrático')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 10  # Número de iterações
test_size = 0.1  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())



model = GaussianNB()
print('\n\n\n\n Naive Bayes')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 10  # Número de iterações
test_size = 0.1  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())






 Linear
Acurácia por repetição: [0.98571429 0.97142857 0.94285714 0.87142857 0.92857143 0.92857143
 0.92857143 0.97142857 0.98571429 0.91428571]
Média da acurácia: 0.9428571428571428
Desvio padrão da acurácia: 0.03440455593940657




 Quadrático
Acurácia por repetição: [0.91428571 0.95714286 0.98571429 0.94285714 0.95714286 0.98571429
 0.95714286 0.95714286 0.97142857 0.92857143]
Média da acurácia: 0.9557142857142857
Desvio padrão da acurácia: 0.02161820850060224




 Naive Bayes
Acurácia por repetição: [0.97142857 0.94285714 0.95714286 0.97142857 1.         0.98571429
 0.97142857 0.95714286 0.97142857 0.97142857]
Média da acurácia: 0.97
Desvio padrão da acurácia: 0.01491472358415793


In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Carregar os dados do arquivo CSV
data = pd.read_csv('Data_CSV/data_Breast_Cancer_Wisconsin.csv')
df = data
# Initialize the imputer
imputer = SimpleImputer(strategy='mean')

# Fit and transform the data
data_filled = imputer.fit_transform(data)
data_filled = pd.DataFrame(data_filled, columns=data.columns)

df = data_filled


# Supondo que a última coluna seja a variável alvo (classe)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Converter rótulos para valores numéricos, se necessário
if y.dtype == 'object':
    y = LabelEncoder().fit_transform(y)

# Criar o modelo Discriminante Linear
model = LinearDiscriminantAnalysis()
print('\n\n\n\n Linear')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 10  # Número de iterações
test_size = 0.1  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())

model = LinearDiscriminantAnalysis()
print('\n\n\n\n Quadrático')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 10  # Número de iterações
test_size = 0.1  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())



model = GaussianNB()
print('\n\n\n\n Naive Bayes')
# Definir o número de repetições do Monte-Carlo Cross-Validation
n_repeats = 10  # Número de iterações
test_size = 0.1  # 10% para teste

# Armazenar as acurácias
scores = []

# Loop para realizar as divisões aleatórias
for _ in range(n_repeats):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=None, stratify=y)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores.append(accuracy_score(y_test, y_pred))

# Exibir os resultados
scores = np.array(scores)
print('Acurácia por repetição:', scores)
print('Média da acurácia:', scores.mean())
print('Desvio padrão da acurácia:', scores.std())





 Linear
Acurácia por repetição: [0.98571429 0.97142857 0.95714286 0.94285714 0.98571429 0.94285714
 0.98571429 0.98571429 0.97142857 0.97142857]
Média da acurácia: 0.9700000000000001
Desvio padrão da acurácia: 0.016225452416572233




 Quadrático
Acurácia por repetição: [0.95714286 0.95714286 0.98571429 1.         0.95714286 0.9
 0.91428571 0.97142857 0.95714286 0.95714286]
Média da acurácia: 0.9557142857142857
Desvio padrão da acurácia: 0.02817583274759432




 Naive Bayes
Acurácia por repetição: [0.95714286 0.97142857 0.92857143 1.         0.94285714 0.98571429
 0.97142857 0.97142857 0.95714286 0.94285714]
Média da acurácia: 0.9628571428571429
Desvio padrão da acurácia: 0.020404081224408142


In [40]:
x_l = scores.mean() - scores.std()/n_repeats* z
x_h = scores.mean() + scores.std()/n_repeats* z

print(x_l,x_h)

0.955075391587556 0.9578826066251342


In [37]:
z=3